In [1]:
import os
rootdir = os.path.abspath("..")
print(rootdir)

C:\Users\kamra\Thesis-code


In [2]:
cancer_type = "bladder"
uuids = os.listdir(os.path.join(rootdir, "data", cancer_type))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\kamra\\Thesis-code\\data\\bladder'

In [3]:
sample_file_path = os.path.join(
    rootdir, "data", cancer_type, uuids[0],
    next(filter( lambda s : s.endswith("tsv"),os.listdir(os.path.join(rootdir, "data", cancer_type, uuids[0])) )),
)
sample_file_path

'/home/bashir/Kamran-Dissertation/data/bladder/65ddebff-ab5c-4d45-967b-74f211274363/8123eafd-3824-45ca-8f6a-76b5aca39d13.rna_seq.augmented_star_gene_counts.tsv'

In [4]:
with open(sample_file_path, "r") as f:
    lines = f.readlines()
lines[:20]

['# gene-model: GENCODE v36\n',
 'gene_id\tgene_name\tgene_type\tunstranded\tstranded_first\tstranded_second\ttpm_unstranded\tfpkm_unstranded\tfpkm_uq_unstranded\n',
 'N_unmapped\t\t\t1538355\t1538355\t1538355\t\t\t\n',
 'N_multimapping\t\t\t3541537\t3541537\t3541537\t\t\t\n',
 'N_noFeature\t\t\t1877607\t21359005\t21477610\t\t\t\n',
 'N_ambiguous\t\t\t3766432\t895946\t884364\t\t\t\n',
 'ENSG00000000003.15\tTSPAN6\tprotein_coding\t2821\t1441\t1380\t48.0510\t17.0271\t19.2561\n',
 'ENSG00000000005.6\tTNMD\tprotein_coding\t0\t0\t0\t0.0000\t0.0000\t0.0000\n',
 'ENSG00000000419.13\tDPM1\tprotein_coding\t1722\t907\t815\t110.2297\t39.0605\t44.1737\n',
 'ENSG00000000457.14\tSCYL3\tprotein_coding\t630\t600\t583\t7.0719\t2.5060\t2.8340\n',
 'ENSG00000000460.17\tC1orf112\tprotein_coding\t422\t485\t517\t5.4615\t1.9353\t2.1887\n',
 'ENSG00000000938.13\tFGR\tprotein_coding\t432\t194\t238\t9.8692\t3.4972\t3.9550\n',
 'ENSG00000000971.16\tCFH\tprotein_coding\t3634\t1802\t1832\t35.1848\t12.4679\t14.1000

In [5]:
import pandas as pd
from tqdm.notebook import tqdm

In [6]:
list_of_dfs = []
for uuid in tqdm(uuids):
    try:
        filepath = os.path.join(rootdir, "data", cancer_type, uuid, 
                                next(filter( lambda s : s.endswith("tsv"),os.listdir(os.path.join(rootdir, "data", cancer_type, uuid)) ))
                               )
        df = pd.read_csv(filepath, sep="\t", skiprows=[0,2,3,4,5]) #, header=["GenID", ""])
        list_of_dfs.append(df)
    except Exception as e:
        print(f"Error with {filepath}\n{e}")

  0%|          | 0/431 [00:00<?, ?it/s]

In [7]:
# Is there a null value in any of the dataframes?
any(
    [df.isna().any().any() for df in list_of_dfs]
)

False

In [8]:
# confirm gene_id to gene_name mapping is unique
id_name_mapping = pd.concat([df[['gene_id', 'gene_name']] for df in list_of_dfs]).drop_duplicates()
id_name_mapping.groupby('gene_id').gene_name.count().reset_index().query('gene_name>1')

,gene_id,gene_name


In [9]:
# How many genes for each gene type
gname_gtype_mapping = pd.concat([df[['gene_name', 'gene_type']] for df in list_of_dfs]).drop_duplicates()
gname_gtype_mapping.groupby('gene_type').gene_name.count().reset_index().sort_values(by='gene_name', ascending=False)

,gene_type,gene_name
22,protein_coding,19938
17,lncRNA,16882
21,processed_pseudogene,10160
38,unprocessed_pseudogene,2612
18,miRNA,1879
30,snRNA,1837
19,misc_RNA,1279
10,TEC,1057
34,transcribed_unprocessed_pseudogene,938
31,snoRNA,793


In [17]:
list_of_dfs[0].tpm_unstranded 

0         48.0510
1          0.0000
2        110.2297
3          7.0719
4          5.4615
           ...   
60655      0.0000
60656      7.6964
60657      0.0000
60658      0.0562
60659      1.1957
Name: tpm_unstranded, Length: 60660, dtype: float64